<a href="https://colab.research.google.com/github/facahto/Kaggle-competition-practice/blob/main/forecasting-store_sales/Stores_Sales_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, numpy as np
import torch
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


label_encoder = preprocessing.LabelEncoder()

Data

In [2]:
# Holiday Event
h_e = pd.read_csv('holidays_events.csv')
display(h_e.head(3))
print()
print(h_e.info())
#display(h_e[h_e.duplicated(keep=False)]) ----- NO DUPLICATED DATA

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB
None


In [3]:
# Oil
oil = pd.read_csv('oil.csv')
oil['dcoilwtico'].fillna(float(oil['dcoilwtico'].mean()), inplace=True)
display(oil.head(3))
print()
print(oil.info())
#display(oil[oil.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,date,dcoilwtico
0,2013-01-01,67.714366
1,2013-01-02,93.140000
2,2013-01-03,92.970000



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB
None


In [4]:
# Stores
stores = pd.read_csv('stores.csv')
display(stores.head(3))
print()
print(stores.info())
#display(stores[stores.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB
None


In [5]:
# Transactions
ts = pd.read_csv('transactions.csv')
display(ts)
print()
print(ts.info())
#display(ts[ts.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB
None


In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
display(train.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1.0,AUTOMOTIVE,0.0,0.0
1,1,2013-01-01,1.0,BABY CARE,0.0,0.0
2,2,2013-01-01,1.0,BEAUTY,0.0,0.0
3,3,2013-01-01,1.0,BEVERAGES,0.0,0.0
4,4,2013-01-01,1.0,BOOKS,0.0,0.0


In [7]:
# Preprocessing train data
train['family']= label_encoder.fit_transform(train['family'])
test['family']= label_encoder.fit_transform(test['family'])

In [8]:
#train = pd.merge(train, ts, how="left", on=["date", "store_nbr"])

In [9]:
nan     = train.isna().sum()
nan_row = train[train.isna().any(axis=1)]
print("Jumlah nilai NaN:", nan)

Jumlah nilai NaN: id             0
date           1
store_nbr      1
family         0
sales          1
onpromotion    1
dtype: int64


In [10]:
display(train)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1.0,0,0.0,0.0
1,1,2013-01-01,1.0,1,0.0,0.0
2,2,2013-01-01,1.0,2,0.0,0.0
3,3,2013-01-01,1.0,3,0.0,0.0
4,4,2013-01-01,1.0,4,0.0,0.0
...,...,...,...,...,...,...
1878526,1878526,2015-11-23,18.0,1,0.0,0.0
1878527,1878527,2015-11-23,18.0,2,1.0,0.0
1878528,1878528,2015-11-23,18.0,3,1887.0,0.0
1878529,1878529,2015-11-23,18.0,4,0.0,0.0


In [11]:
## predict oil price for test dataframe

B = pd.DataFrame(columns=['B'])
valB       = oil['dcoilwtico'][1:]
valB[1218] = 45
B['B']     = valB.values
oil = pd.concat([oil, B],axis = 1)

scaler = MinMaxScaler()
oil[['dcoilwtico', 'B']] = scaler.fit_transform(oil[['dcoilwtico', 'B']])
display(oil)

,date,dcoilwtico,B
0,2013-01-01,0.491820,0.792965
1,2013-01-02,0.792965,0.790951
2,2013-01-03,0.790951,0.792728
3,2013-01-04,0.792728,0.793675
4,2013-01-07,0.793675,0.793794
...,...,...,...
1213,2017-08-25,0.254175,0.239370
1214,2017-08-28,0.239370,0.240081
1215,2017-08-29,0.240081,0.234158
1216,2017-08-30,0.234158,0.249556


In [12]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

train_oil   = oil[:1100]
X_train_oil = train_oil['dcoilwtico'].values.reshape(-1, 1)
y_train_oil = train_oil['B'].values.reshape(-1, 1)

test_oil   = oil[1100:]
X_test_oil = test_oil['dcoilwtico'].values.reshape(-1, 1)
y_test_oil = test_oil['B'].values.reshape(-1, 1)


model_oil = LinearRegression()
model_oil.fit(X_train_oil, y_train_oil)

pred_oil = model_oil.predict(X_test_oil)

mae = MAE(y_test_oil, pred_oil)
rmse = np.sqrt(MSE(y_test_oil, pred_oil))
print("RMSE : % f" %(rmse))
print("MAE : % f" %(mae))



RMSE :  0.050555
MAE :  0.020785


In [13]:
oil_list = []

a = np.array([[y_test_oil[-1][0]]])
for i in range(len(test)):
  a = a.reshape(1, -1)
  k = model_oil.predict(a)
  oil_list.append(k[0][0])
  a = k

test['dcoilwtico'] = oil_list
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day

In [14]:
train = pd.merge(train, oil, how="left", on="date")
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

In [15]:
train['dcoilwtico'].fillna(float(train['dcoilwtico'].mean()), inplace=True)
nan     = train.isna().sum()
nan_row = train[train.isna().any(axis=1)]
train   = train.dropna()
print("Jumlah nilai NaN:", nan)

Jumlah nilai NaN: id                  0
date                1
store_nbr           1
family              0
sales               1
onpromotion         1
dcoilwtico          0
B              538165
year                1
month               1
day                 1
dtype: int64


In [16]:
train = train.drop('B',axis=1)

In [17]:
display(test)

,id,date,store_nbr,family,onpromotion,dcoilwtico,year,month,day
0,3000888,2017-08-16,1,0,0,0.233595,2017,8,16
1,3000889,2017-08-16,1,1,0,0.243995,2017,8,16
2,3000890,2017-08-16,1,2,2,0.254004,2017,8,16
3,3000891,2017-08-16,1,3,20,0.263637,2017,8,16
4,3000892,2017-08-16,1,4,0,0.272908,2017,8,16
...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,28,1,0.510267,2017,8,31
28508,3029396,2017-08-31,9,29,0,0.510267,2017,8,31
28509,3029397,2017-08-31,9,30,1,0.510267,2017,8,31
28510,3029398,2017-08-31,9,31,9,0.510267,2017,8,31


In [18]:
train[['store_nbr','family','sales','onpromotion','month','day']] = scaler.fit_transform(train[['store_nbr','family','sales','onpromotion','month','day']])

display(train)

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,year,month,day
0,0,2013-01-01,0.000000,0.00000,0.000000,0.0,0.491820,2013.0,0.000000,0.000000
1,1,2013-01-01,0.000000,0.03125,0.000000,0.0,0.491820,2013.0,0.000000,0.000000
2,2,2013-01-01,0.000000,0.06250,0.000000,0.0,0.491820,2013.0,0.000000,0.000000
3,3,2013-01-01,0.000000,0.09375,0.000000,0.0,0.491820,2013.0,0.000000,0.000000
4,4,2013-01-01,0.000000,0.12500,0.000000,0.0,0.491820,2013.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
1878525,1878525,2015-11-23,0.320755,0.00000,0.000000,0.0,0.154921,2015.0,0.909091,0.733333
1878526,1878526,2015-11-23,0.320755,0.03125,0.000000,0.0,0.154921,2015.0,0.909091,0.733333
1878527,1878527,2015-11-23,0.320755,0.06250,0.000022,0.0,0.154921,2015.0,0.909091,0.733333
1878528,1878528,2015-11-23,0.320755,0.09375,0.040781,0.0,0.154921,2015.0,0.909091,0.733333


In [ ]:
X = train.drop(['id','date','sales','year'],axis=1)
y = train['sales']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=421)

#model = XGBRegressor(objective='reg:squarederror', n_estimators = 10,learning_rate=0.2, seed = 1230)
#(0.3:0.01),(0.5:0.007),(0.7:0.007)
#model.fit(train_X, train_y)

#pred_xgb_r = model.predict(test_X)

#perbandingan_xgb = pd.DataFrame({
#    'aktual': test_y,
#    'prediksi': pred_xgb_r
#})

#mae = MAE(test_y, pred_xgb_r)
#rmse = np.sqrt(MSE(test_y, pred_xgb_r))
#print("RMSE : % f" %(rmse))
#print("MAE : % f" %(mae))

# Build the ANN model
model = Sequential()

# Add input layer
model.add(Dense(units=64, activation='sigmoid', input_dim=6))

# Add hidden layers
model.add(Dense(units=32, activation='sigmoid'))
model.add(Dense(units=16, activation='sigmoid'))

# Add output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(train_X, train_y, epochs=5, batch_size=32, validation_data=(test_X, test_y))

# Evaluate the model
loss = model.evaluate(test_X, test_y)
print(f"Test Loss: {loss}")


Epoch 1/5
25596/33510 [=====================>........] - ETA: 18s - loss: 3.3548e-04

In [ ]:
test[['store_nbr','family','onpromotion','month','day']] = scaler.fit_transform(test[['store_nbr','family','onpromotion','month','day']])
test_data = test.drop(['id','date','year'],axis=1)
pred_y = model.predict(test_data)
test['sales'] = scaler.inverse_transform(pred_y.reshape(-1, 1))
display(test)

In [ ]:
final_result = test[['id','sales']]
display(final_result)

In [ ]:
final_result.to_csv('final_result.csv', index=False)